## RCL (using PyBLP)

In [1]:
import pandas as pd
import numpy as np
import pyblp

### Data Preparation

In [2]:
df = pd.read_csv('../data/Data.csv', 
                 names=['market_id', 'product_id', 'product_ms',
                        'product_att1', 'product_att2', 'product_att3',
                        'price', 'cost_att1', 'cost_att2', 'cost_att3', 'group'])

In [3]:
df = df.join(
    df.groupby('market_id')[['product_ms']]
        .transform('sum')  # Calculate the mean for each group
        .rename(columns='sum_ms'.format)  # Rename columns 
)

In [4]:
df['og_ms'] = 1 - df['sum_ms'] # generate OG market share 

In [5]:
df['delta'] = np.log(df['product_ms']/df['og_ms'])  # generate delta

### PyBLP

In [6]:
df["supply_instruments3"] = np.square(df["product_att1"])
df["supply_instruments4"] = np.square(df["product_att2"])
df["supply_instruments5"] = np.square(df["product_att3"])
df["demand_instruments3"] = np.square(df["cost_att1"])
df["demand_instruments4"] = np.square(df["cost_att2"])
df["demand_instruments5"] = np.square(df["cost_att3"])

In [14]:
df.rename(columns = {'market_id':'market_ids', 
                     'product_id':'product_ids', 
                     'product_ms':'shares', 
                     'price':'prices', 
                     'cost_att1':'demand_instruments0', 
                     'cost_att2':'demand_instruments1', 
                     'cost_att3':'demand_instruments2',
                     
                    }, inplace = True)

In [15]:
df["firm_ids"] = df["product_ids"]

In [16]:
X1_formulation = pyblp.Formulation('1 + prices + product_att1 + product_att2 + product_att3')
X2_formulation = pyblp.Formulation('0 + prices + product_att1 + product_att2 + product_att3')
product_formulations = (X1_formulation, X2_formulation)
product_formulations

(1 + prices + product_att1 + product_att2 + product_att3,
 prices + product_att1 + product_att2 + product_att3)

In [17]:
mc_integration = pyblp.Integration('monte_carlo', size=5000, specification_options={'seed': 0})
mc_integration

Configured to construct nodes and weights with Monte Carlo simulation with options {seed: 0}.

In [18]:
pr_integration = pyblp.Integration('product', size=5)
pr_integration

Configured to construct nodes and weights according to the level-5 Gauss-Hermite product rule with options {}.

In [19]:
mc_problem = pyblp.Problem(product_formulations, df, integration=mc_integration)
mc_problem

Initializing the problem ...
Initialized the problem after 00:00:00.

Dimensions:
 T    N    F     I      K1    K2    MD 
---  ---  ---  ------  ----  ----  ----
50   970  25   250000   5     4     10 

Formulations:
       Column Indices:           0          1             2             3             4      
-----------------------------  ------  ------------  ------------  ------------  ------------
 X1: Linear Characteristics      1        prices     product_att1  product_att2  product_att3
X2: Nonlinear Characteristics  prices  product_att1  product_att2  product_att3              


Dimensions:
 T    N    F     I      K1    K2    MD 
---  ---  ---  ------  ----  ----  ----
50   970  25   250000   5     4     10 

Formulations:
       Column Indices:           0          1             2             3             4      
-----------------------------  ------  ------------  ------------  ------------  ------------
 X1: Linear Characteristics      1        prices     product_att1  product_att2  product_att3
X2: Nonlinear Characteristics  prices  product_att1  product_att2  product_att3              

In [20]:
pr_problem = pyblp.Problem(product_formulations, df, integration=pr_integration)
pr_problem

Initializing the problem ...
Initialized the problem after 00:00:00.

Dimensions:
 T    N    F     I     K1    K2    MD 
---  ---  ---  -----  ----  ----  ----
50   970  25   31250   5     4     10 

Formulations:
       Column Indices:           0          1             2             3             4      
-----------------------------  ------  ------------  ------------  ------------  ------------
 X1: Linear Characteristics      1        prices     product_att1  product_att2  product_att3
X2: Nonlinear Characteristics  prices  product_att1  product_att2  product_att3              


Dimensions:
 T    N    F     I     K1    K2    MD 
---  ---  ---  -----  ----  ----  ----
50   970  25   31250   5     4     10 

Formulations:
       Column Indices:           0          1             2             3             4      
-----------------------------  ------  ------------  ------------  ------------  ------------
 X1: Linear Characteristics      1        prices     product_att1  product_att2  product_att3
X2: Nonlinear Characteristics  prices  product_att1  product_att2  product_att3              

In [21]:
bfgs = pyblp.Optimization('bfgs', {'gtol': 1e-10})

In [22]:
bfgs

Configured to optimize using the BFGS algorithm implemented in SciPy with analytic gradients and options {gtol: +1.000000E-10}.

In [23]:
results1 = mc_problem.solve(sigma=np.ones((4, 4)), optimization=bfgs)
results1

Solving the problem ...

Nonlinear Coefficient Initial Values:
   Sigma:        prices      product_att1   product_att2   product_att3   |  Sigma Squared:     prices      product_att1   product_att2   product_att3 
------------  -------------  -------------  -------------  -------------  |  --------------  -------------  -------------  -------------  -------------
   prices     +1.000000E+00                                               |      prices      +1.000000E+00  +1.000000E+00  +1.000000E+00  +1.000000E+00
product_att1  +1.000000E+00  +1.000000E+00                                |   product_att1   +1.000000E+00  +2.000000E+00  +2.000000E+00  +2.000000E+00
product_att2  +1.000000E+00  +1.000000E+00  +1.000000E+00                 |   product_att2   +1.000000E+00  +2.000000E+00  +3.000000E+00  +3.000000E+00
product_att3  +1.000000E+00  +1.000000E+00  +1.000000E+00  +1.000000E+00  |   product_att3   +1.000000E+00  +2.000000E+00  +3.000000E+00  +4.000000E+00
Starting optimization ...

 1         15           25           569         1751         0     +9.589416E-02  +3.841193E-02  +1.565832E+00  -6.879673E-01, -4.728172E-01, -9.069014E-01, +9.484524E-01, -6.884400E-01, +6.899227E-01, +6.899957E-02, +3.102255E-01, +2.419819E-01, +4.141474E-01
 1         16           26           555         1709         0     +5.665198E-02  +3.924218E-02  +1.351349E+00  -6.507130E-01, -5.003544E-01, -9.018040E-01, +9.027345E-01, -6.806339E-01, +6.961923E-01, +9.933810E-02, +2.007414E-01, +3.022346E-01, +4.200413E-01
 1         17           27           560         1730         0     +1.954202E-02  +3.710996E-02  +2.470872E-01  -6.403979E-01, -5.100014E-01, -8.851217E-01, +8.805549E-01, -6.657943E-01, +7.107045E-01, +1.330576E-01, +1.847378E-01, +3.067561E-01, +3.892673E-01
 1         18           28           537         1655         0     +7.445565E-03  +1.209645E-02  +1.628603E-01  -6.232320E-01, -5.388518E-01, -8.797901E-01, +8.540195E-01, -6.568180E-01, +7.208212E-01, +1.533191E-

 1         32           54           559         1723         0     +7.330081E-21                 +3.293087E-10  -5.959417E-01, -5.947330E-01, -8.584788E-01, +8.041997E-01, -6.294663E-01, +7.460448E-01, +2.027645E-01, +6.599980E-02, +3.741896E-01, +3.404977E-01
 1         32           55           560         1726         0     +7.323210E-21                 +3.403881E-10  -5.959417E-01, -5.947330E-01, -8.584788E-01, +8.041997E-01, -6.294663E-01, +7.460448E-01, +2.027645E-01, +6.599980E-02, +3.741896E-01, +3.404977E-01
 1         32           56           558         1723         0     +7.318266E-21                 +3.344627E-10  -5.959417E-01, -5.947330E-01, -8.584788E-01, +8.041997E-01, -6.294663E-01, +7.460448E-01, +2.027645E-01, +6.599980E-02, +3.741896E-01, +3.404977E-01
 1         32           57           560         1723         0     +7.319631E-21                 +3.387893E-10  -5.959417E-01, -5.947330E-01, -8.584788E-01, +8.041997E-01, -6.294663E-01, +7.460448E-01, +2.027645E-

Problem Results Summary:
GMM     Objective      Gradient         Hessian         Hessian     Clipped  Weighting Matrix  Covariance Matrix
Step      Value          Norm       Min Eigenvalue  Max Eigenvalue  Shares   Condition Number  Condition Number 
----  -------------  -------------  --------------  --------------  -------  ----------------  -----------------
 2    +9.863134E-22  +7.260811E-11  -3.834998E-04   +4.621128E+01      0      +9.315625E+01      +3.600824E+17  

Cumulative Statistics:
Computation  Optimizer  Optimization   Objective   Fixed Point  Contraction
   Time      Converged   Iterations   Evaluations  Iterations   Evaluations
-----------  ---------  ------------  -----------  -----------  -----------
 00:07:26       No           34           69          37907       117041   

Nonlinear Coefficient Estimates (Robust SEs in Parentheses):
   Sigma:         prices        product_att1     product_att2     product_att3    |  Sigma Squared:      prices        product_att1  